<a href="https://colab.research.google.com/github/cdiegor/MineracaoDeDados/blob/main/Pr%C3%A1tica_2_Introdu%C3%A7%C3%A3o_%C3%A0_Minera%C3%A7%C3%A3o_de_Dados_Regress%C3%A3o_Linear_e_Log%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Regressão Linear e Logística
**Tópicos:** seleção de atributos *(forward/backward)*, engenharia de atributos com **regressão polinomial**, avaliação com validação cruzada.

**Datasets:**  
- **Classificação** — *Breast Cancer Wisconsin* (binário)  
- **Regressão** — *California Housing* (alvo contínuo)

> Objetivo: dominar um ciclo prático de **seleção de atributos** e **engenharia de atributos** evitando vazamento (CV no treino), e comparar modelos baseline vs. versões com seleção/atributos polinomiais.


In [ ]:

# --- Preparação ---------------------------------------------------------------
# !pip install scikit-learn pandas numpy matplotlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations

from sklearn.datasets import load_breast_cancer, fetch_california_housing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score

np.random.seed(7)
plt.rcParams['figure.figsize'] = (7, 4)



## 1) Dados e divisões
Carregaremos dois conjuntos do `scikit-learn` e criaremos *splits* de treino/teste. As seleções por *stepwise* serão **sempre** feitas com **validação cruzada** no **treino**.


In [ ]:

# Classificação (Breast Cancer)
bc = load_breast_cancer(as_frame=True)
Xc = bc.data.copy()
yc = bc.target.copy()

Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    Xc, yc, test_size=0.2, stratify=yc, random_state=42
)

# Regressão (California Housing)
house = fetch_california_housing(as_frame=True)
Xr = house.data.copy()
yr = house.target.copy()

Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    Xr, yr, test_size=0.2, random_state=42
)

Xc_train.shape, Xr_train.shape



## 2) Utilitários — Avaliação por CV em subconjuntos de atributos
As funções abaixo avaliam um conjunto de colunas usando **Pipeline(StandardScaler → Estimador)** e `cross_val_score`.


In [ ]:

def cv_score_for_features(X, y, feature_list, estimator, cv, scoring):
    """Retorna média do score de CV para as colunas em feature_list."""
    Xsub = X[feature_list]
    pipe = Pipeline([('scaler', StandardScaler()), ('est', estimator)])
    scores = cross_val_score(pipe, Xsub, y, cv=cv, scoring=scoring)
    return scores.mean(), scores.std()

def best_next_feature_forward(X, y, current_feats, candidate_feats, estimator, cv, scoring):
    best_feat, best_mean, best_std = None, -np.inf, None
    for f in candidate_feats:
        feats = current_feats + [f]
        mean, std = cv_score_for_features(X, y, feats, estimator, cv, scoring)
        if mean > best_mean:
            best_feat, best_mean, best_std = f, mean, std
    return best_feat, best_mean, best_std

def worst_feature_backward(X, y, current_feats, estimator, cv, scoring):
    """Retorna a feature cuja remoção melhora mais (ou piora menos) o score."""
    baseline_mean, baseline_std = cv_score_for_features(X, y, current_feats, estimator, cv, scoring)
    best_drop, best_mean, best_std = None, baseline_mean, baseline_std
    improved = False
    for f in current_feats:
        feats = [c for c in current_feats if c != f]
        mean, std = cv_score_for_features(X, y, feats, estimator, cv, scoring)
        if mean >= best_mean + 1e-10:
            improved = True
            best_drop, best_mean, best_std = f, mean, std
    return improved, best_drop, best_mean, best_std, baseline_mean, baseline_std



## 3) Seleção *Forward* (adição sequencial)
Iterativamente, adiciona a **próxima** variável que mais melhora o `score` de CV.


In [ ]:

def forward_stepwise(X, y, estimator, cv, scoring, max_features=None, verbose=True):
    candidates = list(X.columns)
    selected = []
    history = []
    k = 0
    while candidates and (max_features is None or k < max_features):
        best_feat, best_mean, best_std = best_next_feature_forward(
            X, y, selected, candidates, estimator, cv, scoring
        )
        selected.append(best_feat)
        candidates.remove(best_feat)
        k += 1
        history.append({'k': k, 'added': best_feat, 'cv_mean': best_mean, 'cv_std': best_std})
        if verbose:
            print(f"[{k}] + {best_feat:>20s}  -> CV {scoring}: {best_mean:.4f} ± {best_std:.4f}")
    return selected, pd.DataFrame(history)

# Exemplo: classificação com ROC-AUC
cv_cls = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
logreg = LogisticRegression(max_iter=1000)
selected_fwd_cls, hist_fwd_cls = forward_stepwise(
    Xc_train, yc_train, logreg, cv=cv_cls, scoring='accuracy', max_features=8, verbose=True
)
hist_fwd_cls.head()



## 4) Seleção *Backward* (remoção sequencial)
Começa com **todas** as variáveis e remove a que **menos contribui**, desde que o `score` **não piore**.


In [ ]:

def backward_stepwise(X, y, estimator, cv, scoring, min_features=1, verbose=True):
    selected = list(X.columns)
    changed = True
    history = []
    step = 0
    while changed and len(selected) > min_features:
        changed, drop_feat, mean, std, base_mean, base_std = worst_feature_backward(
            X, y, selected, estimator, cv, scoring
        )
        step += 1
        history.append({'step': step, 'dropped': drop_feat, 'cv_mean': mean, 'cv_std': std, 'baseline': base_mean})
        if verbose:
            print(f"[{step}] - {str(drop_feat):>20s}  -> CV {scoring}: {mean:.4f} (base {base_mean:.4f})")
        if changed:
            selected.remove(drop_feat)
    return selected, pd.DataFrame(history)

# Exemplo: regressão (RMSE menor é melhor, então usamos neg_mean_squared_error no scoring)
cv_reg = KFold(n_splits=5, shuffle=True, random_state=7)
linreg = LinearRegression()

scoring_rmse = 'neg_root_mean_squared_error'
selected_bwd_reg, hist_bwd_reg = backward_stepwise(
    Xr_train, yr_train, linreg, cv=cv_reg, scoring=scoring_rmse, min_features=4, verbose=True
)
hist_bwd_reg.head()



## 5) Comparação no *hold-out* (teste)
Treinamos modelos com as listas de atributos selecionadas e comparamos no conjunto de **teste**.


In [ ]:

# Classificação - baseline vs forward selecionado
pipe_base_cls = Pipeline([('scaler', StandardScaler()), ('est', LogisticRegression(max_iter=1000))])
pipe_fwd_cls  = Pipeline([('scaler', StandardScaler()), ('est', LogisticRegression(max_iter=1000))])

pipe_base_cls.fit(Xc_train, yc_train)
auc_base = roc_auc_score(yc_test, pipe_base_cls.predict_proba(Xc_test)[:,1])

pipe_fwd_cls.fit(Xc_train[selected_fwd_cls], yc_train)
auc_fwd  = roc_auc_score(yc_test, pipe_fwd_cls.predict_proba(Xc_test[selected_fwd_cls])[:,1])

print(f"Classificação (ROC-AUC teste) -> Baseline (todas): {auc_base:.3f} | Forward({len(selected_fwd_cls)} feats): {auc_fwd:.3f}")


In [ ]:

# Regressão - baseline vs backward selecionado
pipe_base_reg = Pipeline([('scaler', StandardScaler()), ('est', LinearRegression())])
pipe_bwd_reg  = Pipeline([('scaler', StandardScaler()), ('est', LinearRegression())])

pipe_base_reg.fit(Xr_train, yr_train)
pred_base = pipe_base_reg.predict(Xr_test)
rmse_base = np.sqrt(mean_squared_error(yr_test, pred_base))
r2_base   = r2_score(yr_test, pred_base)

pipe_bwd_reg.fit(Xr_train[selected_bwd_reg], yr_train)
pred_bwd = pipe_bwd_reg.predict(Xr_test[selected_bwd_reg])
rmse_bwd = np.sqrt(mean_squared_error(yr_test, pred_bwd))
r2_bwd   = r2_score(yr_test, pred_bwd)

print(f"Regressão (Teste) -> REMQ: baseline={rmse_base:.3f}, backward={rmse_bwd:.3f} | R2: baseline={r2_base:.3f}, backward={r2_bwd:.3f}")



## 6) Engenharia de atributos — Regressão **Polinomial**
Criamos termos polinomiais de grau 2/3 para um subconjunto de colunas (ex.: `Longitude`, `Latitude`, `MedInc`). Avaliamos por CV e no teste.


In [ ]:

poly_cols = ['Longitude', 'Latitude', 'MedInc']
deg = 2  # altere para 3 e compare

pre_poly = ColumnTransformer([('poly', PolynomialFeatures(degree=deg, include_bias=False), poly_cols)],
                             remainder='passthrough')

pipe_poly = Pipeline([('pre', pre_poly), ('scaler', StandardScaler(with_mean=False)), ('est', LinearRegression())])

cv_mse = -cross_val_score(pipe_poly, Xr_train, yr_train, cv=cv_reg, scoring='neg_root_mean_squared_error')
print(f"CV REMQ (polinomial grau {deg}):", cv_mse.mean().round(3))

pipe_poly.fit(Xr_train, yr_train)
pred_poly = pipe_poly.predict(Xr_test)
rmse_poly = np.sqrt(mean_squared_error(yr_test, pred_poly))
r2_poly = r2_score(yr_test, pred_poly)
print(f"Teste -> REMQ={rmse_poly:.3f} | R2={r2_poly:.3f}")



## 7) Interações/polynomial para **Logística**
Termos de interação também podem ajudar na classificação (com cuidado). Exemplo: aplicar `PolynomialFeatures` em um subconjunto e comparar ROC-AUC por CV.


In [ ]:

cls_poly_cols = list(Xc_train.columns[:6])  # exemplo: usar 6 primeiras colunas
deg_cls = 2

pre_poly_cls = ColumnTransformer([('poly', PolynomialFeatures(degree=deg_cls, include_bias=False), cls_poly_cols)],
                                remainder='passthrough')

log_poly = Pipeline([('pre', pre_poly_cls), ('scaler', StandardScaler(with_mean=False)), ('est', LogisticRegression(max_iter=1000))])

cv_poly_cls = cross_val_score(log_poly, Xc_train, yc_train, cv=cv_cls, scoring='accuracy')
print(f"Classificação (CV Accuracy) com termos polinomiais (grau {deg_cls}): {cv_poly_cls.mean():.3f}")
